In [1]:
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import numpy as np
import gc
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import roc_curve, auc
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import preprocessing
from math import e
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
from sklearn import metrics
from keras import backend as K
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import matplotlib.pyplot as plt
from keras.regularizers import l2
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score
from keras.constraints import unit_norm

import proplot as pplt

In [2]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
    
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Name: /physical_device:GPU:0   Type: GPU
Num GPUs Available:  1


In [3]:
#Preprocessing functions
#3D detrend function
def detrend(x:np.ndarray,time:np.ndarray):
        nt,nx,ny = x.shape
        xtemp = x.reshape(nt,nx*ny)
        p = np.polyfit(time, xtemp, deg=3)
        fit = p[0]*(time[:,np.newaxis] **3)+ p[1]*(time[:,np.newaxis]**2) + p[2]*(time[:,np.newaxis]) + p[3]
        return x - fit.reshape(nt,nx,ny)

#1D detrend function
def altdetrend(x:np.ndarray,time:np.ndarray):
        nt = x.shape
        xtemp = x.reshape(nt)
        p = np.polyfit(time, x, deg=1)
        fit = p[0]*(time[:,np.newaxis])+ p[1]
        return x - fit.reshape(nt)
    
def remove_time_mean(x):
        return x - x.mean(dim='time')

def removeSC(x):
        return x.groupby('time.month').apply(remove_time_mean)

# Calculate std normal anomaly
def calStdNorAnom(x):
    a=[]
    for m in np.unique(x.time.dt.month):
        mData=x[x.time.dt.month==m]
        mRolling=mData.rolling(time=31, center=True).mean().bfill(dim="time").ffill(dim="time")
        sRolling=mData.rolling(time=31, center=True).std().bfill(dim="time").ffill(dim="time")
        normData=(mData-mRolling)/sRolling
        a.append(normData)
    combineArray=xr.concat(a,'time')
    outArray=combineArray.sortby('time')
    return outArray

In [4]:
#Open ERA5 Datasets
data=xr.open_dataset('ERA5_Input_Exp8.nc')
#Load in preprocessed data
cres=data.cres_pre
netTOAcs=data.netTOAcs_pre
crel=data.crel_pre

data1=xr.open_dataset('ERA5_Output_PrecipCon.nc')
pr=data1.pr

lat=cres.lat
lon=cres.lon

time=cres.time

#Try just selecting the indian ocean
cres=cres.sel(lon=slice(40,115))
crel=crel.sel(lon=slice(40,115))
netTOAcs=netTOAcs.sel(lon=slice(40,115))

cres=cres.sel(lat=slice(40,-40))
crel=crel.sel(lat=slice(40,-40))
netTOAcs=netTOAcs.sel(lat=slice(40,-40))



In [6]:
#Select Monsoon Months
months=[6,7,8,9]
leadmonths=[6,7,8,9]

#varOut.where(varOut.time.dt.month.isin(months), drop=True) #Change varOut to desired variable
prec=pr.sel(time=pr.time.dt.month.isin(months))
cresIn=cres.sel(time=cres.time.dt.month.isin(leadmonths))
netTOAcsIn=netTOAcs.sel(time=netTOAcs.time.dt.month.isin(leadmonths))
crelIn=crel.sel(time=crel.time.dt.month.isin(leadmonths))

In [7]:
land=data.lsMask
land=land.sel(time=land.time.dt.month.isin(months))

land=land.sel(lon=slice(60,100))
land=land[:,240:321,:] #for some reason slicing latitude is producing Nans so I select lat manually

In [8]:
#Select only the SAM lat,lon range: 60-100E, 10-30N
precip=prec.sel(lon=slice(60,100))
precip=precip[:,240:321,:] #Same manual lat selection
precip=xr.where(land==0,np.nan,precip) #remove oceans, monsoon is defined as only over land 

#Do weighted correction on precipitation
weights=np.cos(np.deg2rad(precip.lat))
prec_index=precip.weighted(weights).mean(dim=('lat','lon'))
prec_index=prec_index*60*60*24 #conversion to mm/day, exluding dividing by rho and multiplying by 1000mm/m

In [9]:
#Remove seasonal cycle
prec_index=removeSC(prec_index)

In [10]:
#Normalize
prec_index=calStdNorAnom(prec_index)

In [11]:
#Detrend
time=prec_index.time
prec_index=prec_index.to_numpy()
time=time.to_numpy()
time=time.astype(int)/10**9

prec_index=altdetrend(prec_index,time)
prec_index=xr.DataArray(prec_index,coords=[time],dims=['time'])


In [12]:
mysd=prec_index.std()
mymean=prec_index.mean()

#test=pd.cut(prec_index, [mymean - mysd* 10000, mymean - mysd * 2,  mymean - mysd, mymean - 0.5*mysd, mymean + 0.5*mysd, mymean + mysd, mymean + mysd *2, mymean + mysd* 10000])

#buckets=pd.Categorical(pd.cut(prec_index, [mymean - mysd* 10000, mymean - mysd * 2,  mymean - mysd, mymean - 0.5*mysd, mymean + 0.5*mysd, mymean + mysd, mymean + mysd *2, mymean + mysd* 10000])).rename_categories(['very very low','very low','low','average','high','very high','very very high'])

buckets=pd.Categorical(pd.cut(prec_index, [mymean - mysd* 10000,  mymean - 0.5*mysd,  mymean + 0.5*mysd, mymean + mysd* 10000])).rename_categories(['low','average','high'])

le = preprocessing.LabelEncoder()
le.fit(buckets)
high=buckets=='high'
average=buckets=='average'
low=buckets=='low'
print(len(np.where(high)[0]),'high')
print(len(np.where(average)[0]),'average')
print(len(np.where(low)[0]),'low')

labelprec=le.transform(buckets)

# convert integers to dummy variables (i.e. one hot encoded)
#nclasses=7
nclasses=3
dummy_y=to_categorical(labelprec,nclasses) #converts to binary

21 high
22 average
20 low


In [13]:
# prepare Data for CNN

# split into train and test
cres_train, cres_test, crel_train, crel_test, netTOAcs_train, netTOAcs_test, y_train, y_test = train_test_split(cresIn, crelIn, netTOAcsIn, dummy_y, test_size=0.2, random_state=42)

#Add extra dimension to data, required for algorithm
crestrain=cres_train.values
crestrain=crestrain[:,:,:,None]

creltrain=crel_train.values
creltrain=creltrain[:,:,:,None]

netTOAcstrain=netTOAcs_train.values
netTOAcstrain=netTOAcstrain[:,:,:,None]

#---------------------------------------------------------
crestest=cres_test.values
crestest=crestest[:,:,:,None]

creltest=crel_test.values
creltest=creltest[:,:,:,None]

netTOAcstest=netTOAcs_test.values
netTOAcstest=netTOAcstest[:,:,:,None]

X_test=np.array([crestest,creltest,netTOAcstest])
X_train=np.array([crestrain,creltrain,netTOAcstrain])

print(X_train.shape)

X_train_reshape = np.einsum('lkija->klija',X_train)
X_train_reshape.shape

X_test_reshape = np.einsum('lkija->klija',X_test)
X_test_reshape.shape

# check for nan
np.isnan(X_test_reshape).any()


(3, 50, 321, 1440, 1)


False

In [14]:
print(y_train.shape)

(50, 3)


In [15]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #To be compatable with CUDA

In [15]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  

kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu', padding='same',  kernel_regularizer=regularizers.l2(l=0.01),
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.4))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same',  kernel_regularizer=regularizers.l2(l=0.01)))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.4))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same',  kernel_regularizer=regularizers.l2(l=0.01)))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.4))
    
    

    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(512, activation='relu'))
    #cnn3.add(Dropout(rate = 0.2))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    adam = keras.optimizers.Adam(lr=0.001) # learning_rate
    #sgd=keras.optimizers.SGD(lr=0.0001)
    #adadelta=keras.optimizers.Adadelta(learning_rate=0.001)
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=15 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test],y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

Epoch 1/15
3/3 [==============================] - 9s 1s/step - loss: 34.5621 - accuracy: 0.2889 - val_loss: 13.7479 - val_accuracy: 0.0000e+00
Epoch 2/15
3/3 [==============================] - 1s 212ms/step - loss: 40.2934 - accuracy: 0.4222 - val_loss: 2.5708 - val_accuracy: 0.8000
Epoch 3/15
3/3 [==============================] - 1s 206ms/step - loss: 14.6923 - accuracy: 0.3556 - val_loss: 1.4660 - val_accuracy: 0.0000e+00
Epoch 4/15
3/3 [==============================] - 1s 205ms/step - loss: 2.9095 - accuracy: 0.3778 - val_loss: 1.4164 - val_accuracy: 0.0000e+00
Epoch 5/15
3/3 [==============================] - 1s 207ms/step - loss: 1.2975 - accuracy: 0.3778 - val_loss: 1.3721 - val_accuracy: 0.2000
Epoch 6/15
3/3 [==============================] - 1s 209ms/step - loss: 1.2727 - accuracy: 0.5111 - val_loss: 1.3567 - val_accuracy: 0.2000
Epoch 7/15
3/3 [==============================] - 1s 212ms/step - loss: 1.3004 - accuracy: 0.3111 - val_loss: 1.3451 - val_accuracy: 0.2000
Epoch 8

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kgemmell/miniconda3/envs/test_gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_951999/1178505133.py", line 50, in <module>
    validation_data=(X_train_reshape[test],y_train[test]))
  File "/home/kgemmell/miniconda3/envs/test_gpu/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/home/kgemmell/miniconda3/envs/test_gpu/lib/python3.7/site-packages/keras/engine/training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "/home/kgemmell/miniconda3/envs/test_gpu/lib/python3.7/site-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/home/kgemmell/miniconda3/envs/test_gpu/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
 

TypeError: object of type 'NoneType' has no len()

In [ ]:
pred = cnn3.predict((X_train_reshape), batch_size=10)

In [ ]:
pred.shape

In [ ]:
predicted = np.argmax ( pred , axis=-1 )

In [ ]:
print(predicted)
y_true=np.argmax (y_train , axis=-1 )
print(y_true)

cnt=0
for i,p in enumerate(predicted):
    if (p==y_true[i]):
        cnt=cnt+1

#print(cnt, 'out of 201')
print(cnt, 'out of 50')

In [ ]:
pred_test = cnn3.predict(X_test_reshape, batch_size=10)
ytestPred=np.argmax ( pred_test , axis=-1 )
ytruePred=np.argmax ( y_test , axis=-1 )
print(ytestPred)
print(ytruePred)

cnt=0
for i,p in enumerate(ytestPred):
    if (p==ytruePred[i]):
        cnt=cnt+1

#print(cnt,'out of 51')
print(cnt,'out of 13')

In [ ]:
#Print the confusion matrix
#print(metrics.confusion_matrix(ytestPred,ytruePred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(ytestPred,ytruePred, digits=3))

In [ ]:
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='test')
plt.legend()
plt.show()

# Models 1,2,3 (change epochs)

In [ ]:
# Fit model to training data

acc_per_fold = []
loss_per_fold = []
fold_no = 1


#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
        
# Define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=10 # best average accuracy and lowest loss in validation data (cross-validation)
        
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=10, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()
    



# Model 4 

In [ ]:
# Fit model to training data

acc_per_fold = []
loss_per_fold = []
fold_no = 1


#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
        
# Define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=10 # best average accuracy and lowest loss in validation data (cross-validation)
        
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=10, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()
    



# Model 5

In [ ]:
# Fit model to training data

acc_per_fold = []
loss_per_fold = []
fold_no = 1


#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
        
# Define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=5, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=10 # best average accuracy and lowest loss in validation data (cross-validation)
        
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=10, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()
    




# Model 6


In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=5, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    #cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    #cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    #cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=10 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

# Model 7, 8 , 9 (change epochs)

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1


#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(16, activation='relu', kernel_regularizer='l2'))
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=10 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs,batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 10

# Model 11

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1


#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 12

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1


#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(8, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=5 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 13, 14, 15, 16, 17

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(512, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=30, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 18, 19 (change dense)

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(128, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 20

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(128, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))

    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(512, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 21

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))

    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(1024, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 22

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    

    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(256, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 23, 24, 25

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.2))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.3))
    

    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(128, activation='relu'))
    cnn3.add(Dropout(rate = 0.4))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=10, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 26

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  
    
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu',padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(256, activation='relu'))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Different Optimizers/ Kernel Regularization 

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  

kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    cnn3 = Sequential()
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu', padding='same',
                    kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                    bias_regularizer=regularizers.l2(1e-4),
                    activity_regularizer=regularizers.l2(1e-5),
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    #cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same'))
    #cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    #cnn3.add(Dropout(rate = 0.1))
    
    #cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    #cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    #cnn3.add(Dropout(rate = 0.1))
    
    #cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    #cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    #cnn3.add(Dropout(rate = 0.1))
    
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dropout(rate = 0.1))
    cnn3.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    
    adam = keras.optimizers.Adam(lr=0.001) # learning_rate
    sgd=keras.optimizers.SGD(lr=0.0001)
    adadelta=keras.optimizers.Adadelta(learning_rate=0.001)
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=10, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test], y_train[test]))
        
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Models 28 - 33

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  

kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l=0.01),
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    #cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same', kernel_regularizer=regularizers.l2(l=0.01)))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    #cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(128, activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    cnn3.add(Dropout(rate = 0.1))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #sgd=keras.optimizers.SGD(lr=0.0001)
    #adadelta=keras.optimizers.Adadelta(learning_rate=0.001)
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test],y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 34

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  

kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    
    cnn3 = Sequential()
    cnn3.add(Conv3D(8, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l=0.01),
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.2))
    
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu',padding='same', kernel_regularizer=regularizers.l2(l=0.01)))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.2))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same', kernel_regularizer=regularizers.l2(l=0.01)))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.2))
    
    cnn3.add(Conv3D(64, kernel_size=3, activation='relu',padding='same', kernel_regularizer=regularizers.l2(l=0.01)))
    cnn3.add(MaxPooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.2))
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(512, activation='relu',kernel_regularizer=l2(0.01)))
    cnn3.add(Dropout(rate = 0.4))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #sgd=keras.optimizers.SGD(lr=0.0001)
    #adadelta=keras.optimizers.Adadelta(learning_rate=0.001)
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=20 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test],y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()

# Model 41

In [ ]:
# Fit model to training data
# # define 10-fold cross validation test harness

acc_per_fold = []
loss_per_fold = []
fold_no = 1

#To limit memory usage and avoid resource exhaustion
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  

kfold = KFold(n_splits=10, shuffle=True, random_state=0)
for train, test in kfold.split(X_train_reshape, y_train):
    
    cnn3 = Sequential()
    cnn3.add(Conv3D(16, kernel_size=3, activation='relu', padding='same',
                    input_shape=(X_train_reshape[train].shape[1],
                                 X_train_reshape[train].shape[2],X_train_reshape[train].shape[3],1)),)
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    cnn3.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    cnn3.add(AveragePooling3D(pool_size=2,padding='same'))
    cnn3.add(Dropout(rate = 0.1))
    
    
    cnn3.add(Flatten())
    
    
    #Dense function adds a fully connected layer
    #Hidden layer
    cnn3.add(Dense(64, activation='relu'))
    cnn3.add(Dropout(rate = 0.2))
    #Output layer
    cnn3.add(Dense(units= nclasses, activation = "softmax")) #units is always equal to number of classes
    
    adam = keras.optimizers.Adam(lr=0.0001) # learning_rate
    #sgd=keras.optimizers.SGD(lr=0.0001)
    #adadelta=keras.optimizers.Adadelta(learning_rate=0.001)
    #Adam-A optimizer method for Stochastic Optimization
    cnn3.compile(optimizer=adam, loss='categorical_crossentropy',  metrics=['accuracy'])
    epochs=50 # best average accuracy and lowest loss in validation data (cross-validation)
    hist = cnn3.fit(X_train_reshape[train], y_train[train],  epochs=epochs, batch_size=20, verbose=1, shuffle=True,
                         validation_data=(X_train_reshape[test],y_train[test]))
    
    # report performance
    scores = cnn3.evaluate(X_train_reshape[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {cnn3.metrics_names[0]} of {scores[0]}; {cnn3.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
    #To prevent resource exhaustion
    gc.collect()
    K.clear_session()